In [54]:
import pandas as pd
import re
import numpy as np
import souper_v2

## Loading base leagues

In [55]:
URLs_whoscored = {'Primera División (ARG)': 'https://www.whoscored.com/Regions/11/Tournaments/68/Argentina-Primera-Division',
                 'Premier League (ENG)':'https://www.whoscored.com/Regions/252/Tournaments/2/England-Premier-League',
                 'Liga NOS (POR)': 'https://www.whoscored.com/Regions/177/Tournaments/21/Portugal-Liga-NOS',
                 'Super Lig (TUR)': 'https://www.whoscored.com/Regions/225/Tournaments/17/Turkey-Super-Lig',
                 'Serie A (ITA)': 'https://www.whoscored.com/Regions/108/Tournaments/5/Italy-Serie-A',
                 'Eredivise (NET)': 'https://www.whoscored.com/Regions/155/Tournaments/13/Netherlands-Eredivisie',
                 'Championship (ENG)': 'https://www.whoscored.com/Regions/252/Tournaments/7/England-Championship',
                 'La Liga (SPA)': 'https://www.whoscored.com/Regions/206/Tournaments/4/Spain-La-Liga',
                 'Premier Liga (RUS)': 'https://www.whoscored.com/Regions/182/Tournaments/77/Russia-Premier-League',
                 'Bundesliga (GER)': 'https://www.whoscored.com/Regions/81/Tournaments/3/Germany-Bundesliga',
                 'Ligue 1 (FRA)': 'https://www.whoscored.com/Regions/74/Tournaments/22/France-Ligue-1',
                 'Bundesliga 2 (GER)': 'https://www.whoscored.com/Regions/81/Tournaments/6/Germany-Bundesliga-II',
                 'La Liga 2 (SPA)': 'https://www.whoscored.com/Regions/206/Tournaments/63/Spain-Segunda-Division',
                 'Super League (CHI)': 'https://www.whoscored.com/Regions/45/Tournaments/162/China-Super-league',
                 'MLS (USA)': 'https://www.whoscored.com/Regions/233/Tournaments/85/USA-Major-League-Soccer'}

In [56]:
URLs_transfermkt = {'Primera División (ARG)': 'https://www.transfermarkt.co.uk/primera-division/startseite/wettbewerb/AR1N',
                 'Premier League (ENG)':'https://www.transfermarkt.co.uk/premier-league/startseite/wettbewerb/GB1',
                 'Liga NOS (POR)': 'https://www.transfermarkt.co.uk/liga-nos/startseite/wettbewerb/PO1',
                 'Super Lig (TUR)': 'https://www.transfermarkt.co.uk/super-lig/startseite/wettbewerb/TR1',
                 'Serie A (ITA)': 'https://www.transfermarkt.co.uk/serie-a/startseite/wettbewerb/IT1',
                 'Eredivise (NET)': 'https://www.transfermarkt.co.uk/eredivisie/startseite/wettbewerb/NL1',
                 'Championship (ENG)': 'https://www.transfermarkt.co.uk/championship/startseite/wettbewerb/GB2',
                 'La Liga (SPA)': 'https://www.transfermarkt.co.uk/laliga/startseite/wettbewerb/ES1',
                 'Premier Liga (RUS)': 'https://www.transfermarkt.co.uk/premier-liga/startseite/wettbewerb/RU1',
                 'Bundesliga (GER)': 'https://www.transfermarkt.co.uk/bundesliga/startseite/wettbewerb/L1',
                 'Ligue 1 (FRA)': 'https://www.transfermarkt.co.uk/ligue-1/startseite/wettbewerb/FR1',
                 'Bundesliga 2 (GER)': 'https://www.transfermarkt.co.uk/2-bundesliga/startseite/wettbewerb/L2',
                 'La Liga 2 (SPA)': 'https://www.transfermarkt.co.uk/laliga2/startseite/wettbewerb/ES2',
                 'Super League (CHI)': 'https://www.transfermarkt.co.uk/chinese-super-league/startseite/wettbewerb/CSL',
                 'MLS (USA)': 'https://www.transfermarkt.co.uk/major-league-soccer/startseite/wettbewerb/MLS1'}

In [57]:
sp = souper_v2.souper()

In [58]:
# Backup of original simplier function to get_soup
def get_soup(URL):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    wait = WebDriverWait(driver, 10)
    driver.get(URL)
    if len(driver.find_elements_by_css_selector("[name=ROBOTS]")) > 0:
        driver.get(URL)
    soup = BeautifulSoup(driver.page_source)
    driver.close()
    return soup

## Base functions

In [59]:
def extract_teams_or_players(soup, league_or_team, whoscore, team):
    
    returner_df = pd.DataFrame()
    links = []
    teams_or_players = []
    league_or_team_list = []
    
    if whoscore:
        
        if team:
            team_pattern = re.compile(r'(?<=\>).+(?=<)')
            link_pattern = re.compile(r'/Team.+(?=" )')

            for each in soup.find_all('a', attrs={'class':'team-link'}):

                try:
                    team_or_player = re.findall(team_pattern, str(each))[0]
                    link = re.findall(link_pattern, str(each))[0]
                    team_or_player = team.strip()
                    link = link.strip()
                    teams_or_players.append(team_or_player)
                    links.append(link)
                    league_or_team_list.append(league_or_team)

                except:
                    None
                    
        else:
            player_pattern = re.compile(r'(?<=\">).+(?= </a>)')
            link_pattern = re.compile(r'(?<=\ href=").+(?=">)')
            
            for each in soup.find_all('a', attrs={'class':'player-link'}):
            
                try:
                    team_or_player = re.findall(player_pattern, str(each))[0]
                    link = re.findall(link_pattern, str(each))[0]
                    team_or_player = team_or_player.strip()
                    link = link.strip()
                    teams_or_players.append(team_or_player)
                    links.append(link)
                    league_or_team_list.append(league_or_team)
                    
                except:
                    None
                
    else:
        
        if team:
            team_pattern = re.compile(r'(?<=\ alt=").+(?=" class)')
            link_pattern = re.compile(r'(?<=\ href=").+[startseite].+(?=/saison_id/)')

            for each in soup.find_all('a', attrs={'class':'vereinprofil_tooltip tooltipstered'}):
            
                if 'startseite' in str(each):

                    try:
                        team_or_player = re.findall(team_pattern, str(each))[0]
                        link = re.findall(link_pattern, str(each))[0]
                        team_or_player = team_or_player.strip()
                        link = link.strip()
                        teams_or_players.append(team_or_player)
                        links.append(link)
                        league_or_team_list.append(league_or_team)

                    except:
                        None
        
        else:
            player_pattern = re.compile(r'(?<=\">).+(?=</a>)')
            link_pattern = re.compile(r'(?<=\ href=").+(?=" id=)')

            for each in soup.find_all('a', attrs={'class': 'spielprofil_tooltip tooltipstered'}):

                try:
                    team_or_player = re.findall(player_pattern, str(each))[0]
                    link = re.findall(link_pattern, str(each))[0]
                    team_or_player = team_or_player.strip()
                    link = link.strip()
                    teams_or_players.append(team_or_player)
                    links.append(link)
                    league_or_team_list.append(league_or_team)

                except:
                    None
    
    if team:
        returner_df['league'] = league_or_team_list
        returner_df['teams'] = teams_or_players
    else:
        returner_df['team'] = league_or_team_list
        returner_df['player'] = teams_or_players
        
    returner_df['link'] = links
    returner_df = returner_df.drop_duplicates(subset='link', keep="last")

    return returner_df

In [60]:
def get_players(teams_df, team=False):
    
    players_detail_df = pd.DataFrame()
    
    for each in list(teams_df.index):
        
        URL = 'https://www.whoscored.com' + teams_df.ws_link[each]
        whoscore = True
        soup = sp.get_soup(URL)
        players = extract_teams_or_players(soup, teams_df.team[each], whoscore, team)
        players_detail_df = pd.concat([players_detail_df, players], axis=0, sort=True)
        print("Complete ws " + teams_df.team[each])
        
        URL = 'https://www.transfermarkt.co.uk' + teams_df.tm_link[each]
        whoscore = False
        soup = sp.get_soup(URL)
        players = extract_teams_or_players(soup, teams_df.team[each], whoscore, team)
        players_detail_df = pd.concat([players_detail_df, players], axis=0, sort=True)
        print("Complete tm " + teams_df.team[each])
        
    return players_detail_df

In [61]:
def get_teams(URLs_dict, whoscore=True, team=True):
    
    teams_detail_df = pd.DataFrame()
    
    for each in range(len(URLs_dict.keys())):
        
        soup = sp.get_soup(list(URLs_dict.values())[each])
        league = list(URLs_dict.keys())[each]
        teams = extract_teams_or_players(soup, league, whoscore, team)
        teams_detail_df = pd.concat([teams_detail_df, teams], axis=0, sort=True)
        
        print("Complete " + league)
        
    return teams_detail_df

## Obtaining teams from transfermkt and whoscored

In [ ]:
teams_transfermkt = get_teams(URLs_transfermkt, whoscore=False)

In [168]:
teams_transfermkt.to_csv(r'/Users/gonzaloferreiro/Documents/GA_Materials/DS_Inmersive/DSI8-lessons/projects/project-capstone/Moving_forward/teams_transfermarket.csv',index=False)

In [ ]:
teams_whoscored = get_teams(URLs_whoscored, whoscore=True)

In [358]:
teams_whoscored.to_csv(r'/Users/gonzaloferreiro/Documents/GA_Materials/DS_Inmersive/DSI8-lessons/projects/project-capstone/Moving_forward/teams_whoscored.csv',index=False)

## Getting unique list of teams

In [63]:
whoscored_teams = pd.read_csv('/Users/gonzaloferreiro/Documents/GA_Materials/DS_Inmersive/DSI8-lessons/projects/project-capstone/Moving_forward/Old and others/teams_whoscored.csv')


In [24]:
transfermkt_teams = pd.read_csv('/Users/gonzaloferreiro/Documents/GA_Materials/DS_Inmersive/DSI8-lessons/projects/project-capstone/Moving_forward/Old and others/teams_transfermarket.csv')


In [25]:
def repetead_words(ws_teams, tm_teams):
    
    repeated = {'word': [], 'league': []}
    leagues = list(ws_teams.league.unique())
    returner = {}
        
    for tm_league in tm_teams.league.unique():
        for tm in tm_teams.teams[tm_teams.league==tm_league]:
            tm_team_words = tm.split()
            repeated['word'] += tm_team_words
            repeated['league'] += [tm_league for i in range(len(tm_team_words))]
    
    for ws_league in ws_teams.league.unique():
        for ws in ws_teams.teams[ws_teams.league==ws_league]:
            ws_team_words = ws.split()
            repeated['word'] += ws_team_words
            repeated['league'] += [ws_league for i in range(len(ws_team_words))]    
    
    rep = pd.DataFrame(repeated)
    
    for each in leagues:
        returner[each] = list(rep.word[rep.league == each].value_counts()[rep.word[rep.league == each].value_counts() > 2].keys())
    
    return returner

In [35]:
transfermkt_teams.teams = transfermkt_teams.teams.map(lambda v: 'Espanyol' if v == 'RCD Espanyol Barcelona' else v)
transfermkt_teams.teams = transfermkt_teams.teams.map(lambda v: 'ManCity' if v == 'Manchester City' else v)
whoscored_teams.teams = whoscored_teams.teams.map(lambda v: 'ManCity' if v == 'Manchester City' else v)
transfermkt_teams.teams = transfermkt_teams.teams.map(lambda v: 'ManUn' if v == 'Manchester United' else v)
whoscored_teams.teams = whoscored_teams.teams.map(lambda v: 'ManUn' if v == 'Manchester United' else v)

In [36]:
def unique_list_of_teams(ws_teams, tm_teams):
    
    merged_df = {'league': [], 'team': [], 'ws_link': [], 'tm_link': []}
    
    rep = repetead_words(ws_teams,tm_teams)
    
    for ws in range(len(ws_teams.teams)):
        ws_team_words = ws_teams.teams[ws].split()
        remove_words_ws = rep[ws_teams.league[ws]]
        ws_team_words = [x for x in ws_team_words if x not in remove_words_ws]
        for tm in range(len(tm_teams.teams)):
            if ws_teams.teams[ws] == tm_teams.teams[tm] and tm_teams.league[tm] == ws_teams.league[ws]:
                merged_df['league'].append(ws_teams.league[ws])
                merged_df['team'].append(ws_teams.teams[ws])
                merged_df['ws_link'].append(ws_teams.links[ws])
                merged_df['tm_link'].append(tm_teams.link[tm])
            else:
                tm_team_words = tm_teams.teams[tm].split()
                remove_words_tm = rep[tm_teams.league[tm]]
                tm_team_words = [x for x in tm_team_words if x not in remove_words_tm]
                for each_word in ws_team_words:
                    if each_word != "" and each_word in tm_team_words:
                        if tm_teams.league[tm] == ws_teams.league[ws]:                        
                            merged_df['league'].append(ws_teams.league[ws])
                            merged_df['team'].append(ws_teams.teams[ws])
                            merged_df['ws_link'].append(ws_teams.links[ws])
                            merged_df['tm_link'].append(tm_teams.link[tm])
    
    merged_df = pd.DataFrame(merged_df)
    merged_df.drop_duplicates(inplace=True)
    merged_df.drop_duplicates(subset='ws_link',keep=False,inplace=True)
    merged_df.drop_duplicates(subset='tm_link',keep=False,inplace=True)
    return merged_df             

In [37]:
final_list_of_teams = unique_list_of_teams(whoscored_teams,transfermkt_teams)

In [38]:
final_list_of_teams.shape

(266, 4)

In [39]:
final_list_of_teams.ws_link.value_counts().head()

/Teams/189/Show/England-Brentford       1
/Teams/1730/Show/Germany-Augsburg       1
/Teams/1120/Show/USA-Colorado-Rapids    1
/Teams/2200/Show/Turkey-Kayserispor     1
/Teams/227/Show/Turkey-Antalyaspor      1
Name: ws_link, dtype: int64

In [40]:
final_list_of_teams.league.value_counts()

Championship (ENG)        24
MLS (USA)                 21
Serie A (ITA)             20
Premier League (ENG)      20
Primera División (ARG)    19
Eredivise (NET)           18
La Liga (SPA)             18
Bundesliga 2 (GER)        17
Ligue 1 (FRA)             17
Premier Liga (RUS)        16
La Liga 2 (SPA)           16
Bundesliga (GER)          15
Liga NOS (POR)            15
Super Lig (TUR)           15
Super League (CHI)        15
Name: league, dtype: int64

In [41]:
final_list_of_teams.to_csv(r'/Users/gonzaloferreiro/Documents/GA_Materials/DS_Inmersive/DSI8-lessons/projects/project-capstone/Moving_forward/final_list_of_teams.csv',index=False)


## Obtaining final list of players

### Initial sampling

In [64]:
final_teams = pd.read_csv('/Users/gonzaloferreiro/Documents/GA_Materials/DS_Inmersive/DSI8-lessons/projects/project-capstone/Moving_forward/final_list_of_teams.csv')


In [65]:
final_teams.shape

(266, 4)

In [66]:
final_teams.head()

,league,team,ws_link,tm_link
0,Primera División (ARG),Estudiantes,/Teams/901/Show/Argentina-Estudiantes,/estudiantes-de-la-plata/startseite/verein/288
1,Primera División (ARG),San Lorenzo,/Teams/906/Show/Argentina-San-Lorenzo,/club-atletico-san-lorenzo-de-almagro/startsei...
2,Primera División (ARG),Gimnasia LP,/Teams/896/Show/Argentina-Gimnasia-LP,/club-de-gimnasia-y-esgrima-la-plata/startseit...
3,Primera División (ARG),Banfield,/Teams/1303/Show/Argentina-Banfield,/club-atletico-banfield/startseite/verein/830
4,Primera División (ARG),Newells Old Boys,/Teams/902/Show/Argentina-Newells-Old-Boys,/club-atletico-newells-old-boys/startseite/ver...


In [67]:
# Sampling list of teams

teams_model = final_teams.sample(150)

### Checking for teams missed in sampling

In [95]:
resting_teams_index = [x for x in list(final_teams.index) if x not in list(teams_model.index)]

In [96]:
resting_teams = final_teams.iloc[resting_teams_index]

In [97]:
resting_teams.shape

(116, 4)

In [98]:
teams_model.shape

(150, 4)

In [99]:
final_teams.shape

(266, 4)

### Resampling some more teams

In [100]:
teams_model_2 = resting_teams.sample(50)

#### Final run to get players

In [ ]:
players_model = get_players(teams_model)

In [ ]:
players_model_2 = get_players(teams_model_2)

#### Checking data obtained

In [71]:
players_model.reset_index(inplace=True, drop=True)

In [72]:
players_model.shape

(8072, 3)

In [73]:
players_model.head()

,link,player,team
0,/Players/26428/Show/Kei-Kamara,Kei Kamara,Colorado Rapids
1,/Players/72738/Show/Danny-Wilson,Danny Wilson,Colorado Rapids
2,/Players/13776/Show/Benny-Feilhaber,Benny Feilhaber,Colorado Rapids
3,/Players/97070/Show/Diego-Rubio,Diego Rubio,Colorado Rapids
4,/Players/126909/Show/Kellyn-Acosta,Kellyn Acosta,Colorado Rapids


In [102]:
players_model_2.reset_index(inplace=True, drop=True)

In [103]:
players_model_2.shape

(2758, 3)

In [104]:
players_model_2.shape

(2758, 3)

#### Final list of players

In [106]:
final_players = pd.concat([players_model,players_model_2], axis=0)

In [107]:
final_players.shape

(10830, 3)

In [108]:
final_players.to_csv(r'/Users/gonzaloferreiro/Documents/GA_Materials/DS_Inmersive/DSI8-lessons/projects/project-capstone/Moving_forward/final_players_pre_cleaning.csv',index=False)


In [167]:
lastname = [x.split()[-1] for x in final_players.player]

In [168]:
final_players['lastname'] = lastname

In [170]:
final_players_ws = final_players[final_players.link.str.contains('Show')]

In [171]:
final_players_tm = final_players[final_players.link.str.contains('profil')]

In [172]:
final_players_tm.head()

,link,player,team,lastname
21,/tim-howard/profil/spieler/4267,T. Howard,Colorado Rapids,Howard
22,/andre-rawls/profil/spieler/354887,A. Rawls,Colorado Rapids,Rawls
23,/danny-wilson/profil/spieler/103819,D. Wilson,Colorado Rapids,Wilson
24,/lalas-abubakar/profil/spieler/482611,L. Abubakar,Colorado Rapids,Abubakar
25,/axel-sjoberg/profil/spieler/113116,A. Sjöberg,Colorado Rapids,Sjöberg


In [173]:
final_players_tm.shape

(5458, 4)

In [174]:
final_players_ws.shape

(5372, 4)

In [197]:
joint_players = pd.merge(final_players_tm, final_players_ws,
                        how='inner', left_on=['lastname','team'], right_on = ['lastname','team'])

In [198]:
joint_players.head()

,link_x,player_x,team,lastname,link_y,player_y
0,/tim-howard/profil/spieler/4267,T. Howard,Colorado Rapids,Howard,/Players/5582/Show/Tim-Howard,Tim Howard
1,/danny-wilson/profil/spieler/103819,D. Wilson,Colorado Rapids,Wilson,/Players/72738/Show/Danny-Wilson,Danny Wilson
2,/axel-sjoberg/profil/spieler/113116,A. Sjöberg,Colorado Rapids,Sjöberg,/Players/270583/Show/Axel-Sjöberg,Axel Sjöberg
3,/kofi-opare/profil/spieler/256357,K. Opare,Colorado Rapids,Opare,/Players/126723/Show/Kofi-Opare,Kofi Opare
4,/deklan-wynne/profil/spieler/298228,D. Wynne,Colorado Rapids,Wynne,/Players/354258/Show/Deklan-Wynne,Deklan Wynne


In [199]:
final_joint_players = joint_players.drop_duplicates(subset=['lastname','team'], keep=False)

In [208]:
final_joint_players.shape

(4189, 5)

In [200]:
final_joint_players.drop('player_x',axis=1,inplace=True)

In [201]:
final_joint_players.columns = ['link_tm','team','lastname','link_ws','player']

In [202]:
final_joint_players.head()

,link_tm,team,lastname,link_ws,player
0,/tim-howard/profil/spieler/4267,Colorado Rapids,Howard,/Players/5582/Show/Tim-Howard,Tim Howard
1,/danny-wilson/profil/spieler/103819,Colorado Rapids,Wilson,/Players/72738/Show/Danny-Wilson,Danny Wilson
2,/axel-sjoberg/profil/spieler/113116,Colorado Rapids,Sjöberg,/Players/270583/Show/Axel-Sjöberg,Axel Sjöberg
3,/kofi-opare/profil/spieler/256357,Colorado Rapids,Opare,/Players/126723/Show/Kofi-Opare,Kofi Opare
4,/deklan-wynne/profil/spieler/298228,Colorado Rapids,Wynne,/Players/354258/Show/Deklan-Wynne,Deklan Wynne


In [204]:
final_joint_players = final_joint_players[['player','lastname','team','link_tm','link_ws']]

In [205]:
final_joint_players.head()

,player,lastname,team,link_tm,link_ws
0,Tim Howard,Howard,Colorado Rapids,/tim-howard/profil/spieler/4267,/Players/5582/Show/Tim-Howard
1,Danny Wilson,Wilson,Colorado Rapids,/danny-wilson/profil/spieler/103819,/Players/72738/Show/Danny-Wilson
2,Axel Sjöberg,Sjöberg,Colorado Rapids,/axel-sjoberg/profil/spieler/113116,/Players/270583/Show/Axel-Sjöberg
3,Kofi Opare,Opare,Colorado Rapids,/kofi-opare/profil/spieler/256357,/Players/126723/Show/Kofi-Opare
4,Deklan Wynne,Wynne,Colorado Rapids,/deklan-wynne/profil/spieler/298228,/Players/354258/Show/Deklan-Wynne


In [234]:
sampled_players = final_joint_players.sample(1200, random_state=12)

In [206]:
final_joint_players.to_csv(r'/Users/gonzaloferreiro/Documents/GA_Materials/DS_Inmersive/DSI8-lessons/projects/project-capstone/Moving_forward/final_players_post_cleaning.csv',index=False)


In [236]:
sampled_players.to_csv(r'/Users/gonzaloferreiro/Documents/GA_Materials/DS_Inmersive/DSI8-lessons/projects/project-capstone/Moving_forward/final_list_of_sampled_players.csv',index=False)